# Pincone DB (프리티어 사용)

In [1]:
!pip install pinecone

In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")
index_name = "reviewtest"

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',  
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "reviewtest",
    "metric": "cosine",
    "host": "reviewtest-7qjn04l.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [4]:
reviews = [
    {"id": "1", "text": "이 제품 정말 좋아요! 성능이 만족스러워요!", "sentiment": "positive"},
    {"id": "2", "text": "배송이 빠르고 포장도 꼼꼼했어요. 대만족입니다!", "sentiment": "positive"},
    {"id": "3", "text": "디자인도 예쁘고 사용법도 간편해서 마음에 들어요.", "sentiment": "positive"},
    {"id": "4", "text": "가격 대비 훌륭한 제품이에요. 추천합니다.", "sentiment": "positive"},
    {"id": "5", "text": "친구에게 추천받았는데 역시 좋아요!", "sentiment": "positive"},
    {"id": "6", "text": "생각보다 훨씬 좋네요. 기대 이상이에요.", "sentiment": "positive"},
    {"id": "7", "text": "기능이 다양하고 성능도 안정적이에요.", "sentiment": "positive"},
    {"id": "8", "text": "고급스러운 디자인에 만족했어요.", "sentiment": "positive"},
    {"id": "9", "text": "재구매 의사 있습니다. 너무 만족해요.", "sentiment": "positive"},
    {"id": "10", "text": "AS도 친절하고 전반적으로 만족스러워요.", "sentiment": "positive"},
    {"id": "11", "text": "하루 만에 고장났어요. 정말 실망입니다.", "sentiment": "negative"},
    {"id": "12", "text": "설명서도 없고 조립도 어렵네요.", "sentiment": "negative"},
    {"id": "13", "text": "품질이 너무 안 좋아요. 돈 아까워요.", "sentiment": "negative"},
    {"id": "14", "text": "배터리가 금방 닳고 발열도 심해요.", "sentiment": "negative"},
    {"id": "15", "text": "광고와 너무 달라요. 비추천합니다.", "sentiment": "negative"},
    {"id": "16", "text": "서비스도 불친절하고 반품도 어렵네요.", "sentiment": "negative"},
    {"id": "17", "text": "기대 이하의 성능이에요. 실망했어요.", "sentiment": "negative"},
    {"id": "18", "text": "가격만 비싸고 성능은 형편없어요.", "sentiment": "negative"},
    {"id": "19", "text": "처리 속도가 느려서 답답합니다.", "sentiment": "negative"},
    {"id": "20", "text": "디자인은 좋지만 성능은 엉망이에요.", "sentiment": "negative"}
]

In [5]:
# Pinecone index 및 임베딩 모델 ㄹ로드
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key="")
idx = pc.Index(index_name)

model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([
        (
            review['id'],
            review_embed,
            {
                'text': review['text'],
                'sentiment': review['sentiment']
            }
        )
    ])

### 리뷰 데이터 조회

In [9]:
query_text = "이 노트북 성능은 괜찮은가요?"
query_embed = model.encode(query_text).tolist()

result = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)

result

{'matches': [{'id': '17',
              'metadata': {'sentiment': 'negative',
                           'text': '기대 이하의 성능이에요. 실망했어요.'},
              'score': 0.768254757,
              'values': []},
             {'id': '2',
              'metadata': {'sentiment': 'positive',
                           'text': '배송이 빠르고 포장도 꼼꼼했어요. 대만족입니다!'},
              'score': 0.717725635,
              'values': []},
             {'id': '20',
              'metadata': {'sentiment': 'negative',
                           'text': '디자인은 좋지만 성능은 엉망이에요.'},
              'score': 0.711059809,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [10]:
for review in result['matches']:
    metadata = review['metadata']
    print(metadata['text'], f"(감성: {metadata['sentiment']})")

기대 이하의 성능이에요. 실망했어요. (감성: negative)
배송이 빠르고 포장도 꼼꼼했어요. 대만족입니다! (감성: positive)
디자인은 좋지만 성능은 엉망이에요. (감성: negative)


### 리뷰 감성 분석

In [11]:
query_text = "베터리도 빨리 닳고 벽돌보다 무거워요, 비추천"
query_embed = model.encode(query_text).tolist()

result = idx.query(
    vector=query_embed,
    top_k=5,
    include_metadata=True
)

result

{'matches': [{'id': '10',
              'metadata': {'sentiment': 'positive',
                           'text': 'AS도 친절하고 전반적으로 만족스러워요.'},
              'score': 0.90022248,
              'values': []},
             {'id': '1',
              'metadata': {'sentiment': 'positive',
                           'text': '이 제품 정말 좋아요! 성능이 만족스러워요!'},
              'score': 0.841893137,
              'values': []},
             {'id': '14',
              'metadata': {'sentiment': 'negative',
                           'text': '배터리가 금방 닳고 발열도 심해요.'},
              'score': 0.836007416,
              'values': []},
             {'id': '4',
              'metadata': {'sentiment': 'positive',
                           'text': '가격 대비 훌륭한 제품이에요. 추천합니다.'},
              'score': 0.788865268,
              'values': []},
             {'id': '16',
              'metadata': {'sentiment': 'negative',
                           'text': '서비스도 불친절하고 반품도 어렵네요.'},
              'score': 0.773039579,
         

In [13]:
sentiment_counts = {
    "positive": 0,
    'negative': 0
}

for review in result['matches']:
    metadata = review['metadata']["sentiment"]
    sentiment_counts[metadata] += 1

print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: positive
